In [25]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import networkx as nx

In [28]:
dialogues_df = pd.read_csv("../data/dialouges_over_time.csv")
characters_df = pd.read_csv("../data/characters_w_map.csv")
end_sentiment_df = pd.read_csv("../data/end_sentiment.csv")
affiliations_df = pd.read_csv("../data/Affiliation_mortality.csv")



In [16]:
# Fix the DataFrame by converting string representations of lists into actual lists of strings
import ast

def fix_affiliations(entry):
    """
    Converts a string representation of a list into an actual list of strings.
    """
    if isinstance(entry, str):
        try:
            # Safely evaluate the string as a Python literal
            return ast.literal_eval(entry)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if the conversion fails
    return entry  # Return the entry unchanged if it's already a list

# Apply the fix to the 'Affiliation(s)' column
characters_df['Affiliation(s)'] = characters_df['Affiliation(s)'].apply(fix_affiliations)

In [50]:
# Count the number of lines for each character
line_counts = dialogues_df['mapped'].value_counts()

# Get the top 20 characters with the most lines
top_100_characters = line_counts.head(99).reset_index()

# Rename columns for clarity
top_100_characters.columns = ['character', 'line_count']

characters_top = characters_df.loc[characters_df['mapped'].isin(top_100_characters['character'])]
end_sentiment_top = end_sentiment_df.loc[end_sentiment_df['speaker'].isin(top_100_characters['character'])]
dialogues_top = dialogues_df.loc[dialogues_df['mapped'].isin(top_100_characters['character'])]

In [52]:
top_100_characters

,character,line_count
0,tyrion lannister,1752
1,jon snow,1134
2,daenerys targaryen,967
3,cersei lannister,940
4,jaime lannister,931
...,...,...
94,rast,31
95,meryn trant,31
96,jory cassel,31
97,anguy,30


In [55]:
# Clear the graph and re-check the code for correctness
G = nx.Graph()

# Add nodes for all characters
for _, row in characters_top.iterrows():
    character = row['Name']
    G.add_node(character)  # Add the character as a node

# add edges between characters who share at least one affiliation
for i, row in characters_top.iterrows():
    character = row['Name']
    affiliations = row['Affiliation(s)']
    for j, other_row in characters_top.iterrows():
        other_character = other_row['Name']
        other_affiliations = other_row['Affiliation(s)']
        if character != other_character and bool(set(affiliations).intersection(other_affiliations)):
            print(f"Adding edge between {character} and {other_character}")
            G.add_edge(character, other_character)




Adding edge between Anguy and Lysa Arryn
Adding edge between Anguy and Robin Arryn
Adding edge between Anguy and Petyr Baelish
Adding edge between Anguy and Gendry Baratheon
Adding edge between Anguy and Joffrey Baratheon
Adding edge between Anguy and Myrcella Baratheon
Adding edge between Anguy and Renly Baratheon
Adding edge between Anguy and Robert Baratheon
Adding edge between Anguy and Selyse Baratheon
Adding edge between Anguy and Shireen Baratheon
Adding edge between Anguy and Tommen Baratheon
Adding edge between Anguy and Roose Bolton%27s son
Adding edge between Anguy and Ramsay Bolton
Adding edge between Anguy and Roose Bolton
Adding edge between Anguy and Bronn
Adding edge between Anguy and Jory Cassel
Adding edge between Anguy and Lady Crane
Adding edge between Anguy and Craster
Adding edge between Anguy and Xaro Xhoan Daxos
Adding edge between Anguy and Beric Dondarrion
Adding edge between Anguy and Doreah
Adding edge between Anguy and Walder Frey
Adding edge between Anguy 

In [54]:
characters_top

,Name,Born,Died,House(s),Affiliation(s),Title(s),Culture,Father,Mother,Sibling(s),...,Religion,Issue,Also known as,Personal arms,Species,Affiliation,Lover(s),Family,Dead,mapped
1,Anguy,"Dornish Marches , the Stormlands",NaN,NaN,['Brotherhood Without Banners'],NaN,Marcher,NaN,NaN,NaN,...,Lord of Light,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,anguy
2,Lysa Arryn,"Lysa Tully Riverrun , the Riverlands","301 AC The Eyrie , the Vale of Arryn",Tully Arryn (by marriage) Baelish (by marriage),['Unknown'],NaN,Rivermen,{ Hoster Tully },{ Minisa Whent },NaN,...,Faith of the Seven,Robin Arryn,NaN,NaN,NaN,NaN,NaN,NaN,1,lysa arryn
3,Robin Arryn,"289 AC (age 16) Red Keep , King's Landing",NaN,Arryn,['Royce'],Lord of the Eyrie Defender of the Vale Warden ...,Valemen,{ Jon Arryn },{ Lysa Tully },NaN,...,NaN,NaN,Sweetrobin,NaN,NaN,NaN,NaN,NaN,0,robin arryn
5,Petyr Baelish,"The Fingers , the Vale of Arryn","304 AC Winterfell , the North",Baelish,"['Tully', 'Arryn', 'Baratheon', 'Stark']",Master of Coin (formerly) Lord of Harrenhal Lo...,Valemen,{ Lord Baelish },NaN,NaN,...,NaN,NaN,Littlefinger,"Yellow, a black mockingbird",NaN,NaN,NaN,NaN,1,petyr baelish
6,Gendry Baratheon,"Gendry Flea Bottom , King's Landing",NaN,Baratheon ( legitimized ),"['Tobho Mott', 'Targaryen']",Lord of Storm's End Lord Paramount of the Stor...,Crownlander,{ Robert I Baratheon },NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Arya Stark,NaN,0,gendry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Olenna Tyrell,"Olenna Redwyne The Arbor , the Reach","304 AC Highgarden , the Reach",Redwyne Tyrell (by marriage),['Targaryen'],NaN,Reach,NaN,NaN,Viola Redwyne,...,NaN,{ Mace Tyrell },Queen of Thorns,NaN,NaN,NaN,NaN,NaN,1,olenna
287,Varys,Lys,"305 AC Dragonstone , the Crownlands",NaN,"['Targaryen', 'Baratheon', 'Small Council']",Master of Whisperers,Lysene,NaN,NaN,NaN,...,NaN,NaN,The Spider,NaN,NaN,NaN,NaN,NaN,1,varys
289,Waif,NaN,303 AC in Braavos Killed by Arya Stark with Ne...,NaN,['Faceless Men'],NaN,Braavosi,NaN,NaN,NaN,...,Many-Faced God,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,waif
301,Ygritte,Beyond the Wall,"301 AC Castle Black , the Wall",NaN,['Mance Rayder'],NaN,Free Folk,NaN,NaN,NaN,...,Old Gods,NaN,NaN,NaN,NaN,NaN,Jon Snow,NaN,1,ygritte
